In [2]:
import pandas as pd
import numpy as np

import os

In [3]:
# Thiết lập tùy chọn hiển thị của pandas
pd.set_option('display.max_columns', None)  # Hiển thị tất cả các cột
pd.set_option('display.max_rows', None)     # Hiển thị tất cả các hàng

# Đặt lại tùy chọn hiển thị về giá trị mặc định nếu cần
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')

In [5]:
def check_missing_data(df, columns):
    """
    Kiểm tra dữ liệu bị thiếu trong các cột được chỉ định của DataFrame.
    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        columns (list): Danh sách tên các cột cần kiểm tra.
    Returns:
        DataFrame: DataFrame chứa các dòng dữ liệu có dữ liệu bị thiếu trong các cột được chỉ định.
    """
    missing_data = df[df[columns].isnull().any(axis=1)]
    if missing_data.empty:
        print("Không có dữ liệu bị thiếu trong các cột:", ", ".join(columns))
    else:
        print("Số lượng dòng dữ liệu bị thiếu trong các cột", ", ".join(columns), "là:", missing_data.shape[0])
        print("Các dòng dữ liệu bị thiếu trong các cột", ", ".join(columns), "là:")
        # print(missing_data)
    return missing_data

def check_duplicate_data(df):
    """
    Hàm để kiểm tra và theo dõi dữ liệu trùng lặp trong DataFrame.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.

    Returns:
        DataFrame: DataFrame chứa các dòng dữ liệu trùng lặp.
    """
    # Kiểm tra dữ liệu trùng lặp
    duplicate_data = df[df.duplicated()]

    # In thông báo nếu không có dữ liệu trùng lặp
    if duplicate_data.empty:
        print("Không có dữ liệu trùng lặp.")
    else:
      print("Những dữ liệu trùng lặp: ", duplicate_data)
    return duplicate_data


def count_date_formats(df, column, print_result=False):
    """
    Hàm để kiểm tra và theo dõi số lượng định dạng của dữ liệu ngày tháng trong một cột.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        column (str): Tên của cột cần kiểm tra.
        print_result (bool, optional): True nếu muốn in kết quả ra màn hình, False nếu không (mặc định là False).

    Returns:
        dict: Một từ điển với các định dạng ngày tháng làm khóa và số lượng dòng có mỗi định dạng làm giá trị.
    """
    # Chuyển đổi các giá trị của cột thành chuỗi định dạng ngày tháng
    date_strings = df[column].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, pd.Timestamp) else x)

    # Đếm số lượng giá trị duy nhất cho mỗi định dạng ngày tháng
    format_counts = date_strings.value_counts()

    # Chuyển đổi kết quả thành từ điển
    format_counts_dict = format_counts.to_dict()

    # In kết quả nếu cần
    if print_result:
        print("Số lượng định dạng của dữ liệu ngày tháng:")
        for format, count in format_counts_dict.items():
            print(f"{format}: {count} dòng")

    return format_counts_dict

def score_labeling(df, column_name, bins=[0, 4, 5, 6, 7, 8, 9, 10], labels=['kém', 'yếu', 'trung bình', 'trung bình khá', 'khá', 'giỏi', 'xuất sắc']):
    """
    Hàm để chuyển điểm thành biến phân loại kết quả học tập và trả về cột mới.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        column_name (str): Tên cột chứa điểm cần phân loại.
        bins (list): Danh sách khoảng giá trị cho biến phân loại. Mặc định là [0, 4, 5, 6, 7, 8, 9, 10].
        labels (list): Danh sách nhãn cho các khoảng giá trị. Mặc định là ['kém', 'yếu', 'trung bình', 'trung bình khá', 'khá', 'giỏi', 'xuất sắc'].

    Returns:
        Series: Cột mới chứa biến phân loại.
    """
    # Thêm cột mới vào DataFrame chứa biến phân loại
    df['{}_xl'.format(column_name)] = pd.cut(df[column_name], bins=bins, labels=labels, right=False)
    return df['{}_xl'.format(column_name)]

## Data Quality Rules

In [6]:
# Hàm đánh giá tính đầy đủ (Completeness)
def check_completeness(row, required_columns):
    missing_count = row[required_columns].isna().sum()
    return 1 if missing_count == 0 else 0

# Hàm đánh giá tính nhất quán (Consistency)
def check_consistency(row):
    try:
        # Kiểm tra định dạng ngày tháng
        pd.to_datetime(row['ngaysinh'])
        # Kiểm tra các giá trị hợp lệ của giới tính
        if row['gioitinh'] not in ['Nam', 'Nữ']:
            return 0

        return 1
    except:
        return 0

# Hàm đánh giá tính kịp thời (Timeliness)
def check_timeliness(row, current_year):
    try:
        # Giả sử dữ liệu cập nhật mỗi năm và năm học trong dữ liệu không quá cũ
        year_of_birth = pd.to_datetime(row['ngaysinh']).year
        if year_of_birth <= current_year:
            return 1
        else:
            return 0
    except:
        return 0

# Hàm đánh giá tính hợp lệ (Validity)
def check_validity(row):
    try:
        # Kiểm tra các giá trị hợp lệ của giới tính
        if row['gioitinh'] not in ['Nam', 'Nữ']:
            return 0
        
        # Kiểm tra định dạng ngày tháng
        pd.to_datetime(row['ngaysinh'])
        
        return 1
    except:
        return 0

# Hàm đánh giá tính duy nhất (Uniqueness)
def check_uniqueness(df, column):
    duplicate_count = df.duplicated(subset=[column]).sum()
    return 1 if duplicate_count == 0 else 0

# Giả sử dssinhvien là DataFrame của bạn
# current_year là năm hiện tại
current_year = 2023

## Connect to dataset: dssinhvien_Thu.xlsx

In [16]:
# Loading the New Customer Data from the excel file
df = pd.read_excel('data/dssinhvien_Thu.xlsx')

In [17]:
# Checking first 5 records from Data
df.head(5)

,STT,masv,ngaysinh,gioitinh,noisinh,lopsh,khoa,hedt,nganh,tinhtrang,dantoc,tongiao,xuatthan,doituong,doituong_chitiet,khuvuc,vungmien,ngayvaodoan,ngayvaodang,diachi_phuongxa,diachi_quanhuyen,diachi_tinhtp,diachi_tamtru,diachi_tamtru_quanhuyen,diachi_tamtru_tinhtp,thanhtich_khenthuong,kyluat,quatrinh_congtac,chucvu_caonhat,bhyt,baoho_hoten,baoho_diachi,ghichu,baotin_diachi,bietdentu,bietdentu_other,lydohoc,lydohoc_other
0,1,6520001,05/10/1988,Nam,Sông Bé,TNTH0002,KHMT,CQUI,D480101,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,15/03/2004,NaN,NaN,Thị xã Bình Long,Tỉnh Bình Phước,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6520002,22/10/1988,Nữ,Tỉnh Bến Tre,TNTH0002,HTTT,CQUI,D480104,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,19/05/2004,NaN,Phường 5,Thành phố Bến Tre,Tỉnh Bến Tre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6520004,27/11/1987,Nữ,Thành phố Hồ Chí Minh,TNTH0002,HTTT,CQUI,D480104,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,11/01/2003,NaN,Xã An Ninh Tây,Huyện Đức Hòa,Tỉnh Long An,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,6520005,08/09/1988,Nam,Tỉnh Bình Thuận,TNTH0002,KHMT,CQUI,D480101,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,23/03/2003,NaN,Phường Lạc Đạo,Thành phố Phan Thiết,Tỉnh Bình Thuận,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,6520006,21/01/1988,Nam,Tỉnh Đồng Nai,TNTH0002,KHMT,CQUI,D480101,Đã tốt nghiệp,Kinh,Không,NaN,NaN,NaN,NaN,NaN,26/03/2003,NaN,Phường Xuân An,Thành phố Long Khánh,Tỉnh Đồng Nai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# 2. Khám phá dữ liệu ban đầu
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17925 entries, 0 to 17924
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   STT                      17925 non-null  int64 
 1   masv                     17925 non-null  int64 
 2   ngaysinh                 17900 non-null  object
 3   gioitinh                 17925 non-null  object
 4   noisinh                  17437 non-null  object
 5   lopsh                    17925 non-null  object
 6   khoa                     17925 non-null  object
 7   hedt                     17925 non-null  object
 8   nganh                    17925 non-null  object
 9   tinhtrang                17925 non-null  object
 10  dantoc                   17917 non-null  object
 11  tongiao                  17917 non-null  object
 12  xuatthan                 16358 non-null  object
 13  doituong                 635 non-null    object
 14  doituong_chitiet         524 non-null 

## Data Pre-processing

### 'masv'
- Kiểm tra có mã số sinh viên nào bị trùng lặp hay không

In [19]:
# Kiểm tra dữ liệu trùng lặp trong một cột cụ thể
column_name = 'masv'  # Thay 'ten_cot' bằng tên cột bạn muốn kiểm tra

duplicate_values = df[df.duplicated(subset=[column_name], keep=False)]
if duplicate_values.empty:
    print("Không có dữ liệu trùng lặp trong cột", column_name)
else:
    print("Dữ liệu trùng lặp trong cột", column_name, "là:")
    print(duplicate_values)

Không có dữ liệu trùng lặp trong cột masv


### 'ngaysinh' -> 'ngaysinh_prc'
- chuyển về định dạng dữ liệu ngày tháng: datetime
- kiểm tra và in ra các dữ liệu ngày sinh không đồng dạng.
- tách thành 'ngày' - 'tháng' - 'năm'

In [44]:
# Kiểm tra định dạng của dữ liệu ngày sinh
df['ngaysinh_prc'] = pd.to_datetime(df['ngaysinh'], format='%d/%m/%Y', errors='coerce')

# Kiểm tra xem có dữ liệu nào không phù hợp với định dạng không
invalid_dates = df[df['ngaysinh_prc'].isna()]
if invalid_dates.empty:
    print("Tất cả các dữ liệu ngày sinh đều có đúng định dạng '%d/%m/%Y'.")
else:
    print("Các dữ liệu ngày sinh sau không phù hợp với định dạng '%d/%m/%Y':")
    print(invalid_dates.shape)
    print(invalid_dates)

Các dữ liệu ngày sinh sau không phù hợp với định dạng '%d/%m/%Y':
(36, 40)
       STT      masv  ngaysinh gioitinh          noisinh     lopsh    khoa  \
615    616   7520090       NaN      Nam              NaN  TNTH0002  MMT&TT   
671    672   7520149       NaN      Nam   Tỉnh Khánh Hòa  TNTH0002    HTTT   
832    833   7520318       NaN       Nữ              NaN  TNTH0002    HTTT   
1088  1089   8520007       NaN       Nữ              NaN  TNTH0002    HTTT   
1123  1124   8520044       NaN      Nam  Tỉnh Quảng Ngãi  TNTH0002    CNPM   
1137  1138   8520058  1991-1-1       Nữ              NaN  TNTH0002    HTTT   
1142  1143   8520063       NaN       Nữ              NaN  TNTH0002    HTTT   
1144  1145   8520065  1991-1-1       Nữ              NaN  TNTH0002    HTTT   
1296  1297   8520237       NaN       Nữ              NaN  TNTH0002    HTTT   
1347  1348   8520299       NaN       Nữ              NaN  TNTH0002    HTTT   
1388  1389   8520342       NaN       Nữ              NaN  TNTH0002 

### ngaysinh -> 'birth_day-month-year'


In [51]:
# Tạo các thuộc tính ngày, tháng, năm từ 'ngaysinh_prc'
df['birth_day'] = df['ngaysinh_prc'].dt.day
df['birth_month'] = df['ngaysinh_prc'].dt.month
df['birth_year'] = df['ngaysinh_prc'].dt.year

### 'noisinh' -> 'birth_place'
Đồng bộ các dữ liệu nơi sinh: 
 - lower case
 - xóa bỏ ký tự đặc biệt
 - đồng bộ dữ liệu thành phố: thành phố = tp
 - đồng bộ dữ liệu có "tỉnh" và không có "tỉnh": tỉnh bà rịa vũng tàu = bà rịa vũng tàu
 

In [21]:
df['noisinh'].unique()

array(['Sông Bé', 'Tỉnh Bến Tre', 'Thành phố Hồ Chí Minh',
       'Tỉnh Bình Thuận', 'Tỉnh Đồng Nai', 'Tỉnh Kiên Giang',
       'Tỉnh Lâm Đồng', nan, 'Cửu Long', 'Tỉnh Tây Ninh',
       'Tỉnh Quảng Ngãi', 'Tỉnh Thanh Hóa', 'Tỉnh Long An',
       'Tỉnh Đồng Tháp', 'Tỉnh Nghệ An', 'Tỉnh Hà Tĩnh',
       'Tỉnh Bà Rịa - Vũng Tàu', 'Thành phố Hải Phòng', 'Tỉnh Quảng Nam',
       'Tỉnh Nam Định', 'Tỉnh Khánh Hòa', 'Tỉnh Tiền Giang',
       'Tỉnh Bắc Giang', 'Tỉnh Quảng Bình', 'Tỉnh Đắk Lắk',
       'Tỉnh Gia Lai', 'Nghĩa Bình', 'Thành phố Hà Nội',
       'Thành phố Đà Nẵng', 'Tỉnh Quảng Trị', 'Tỉnh Thừa Thiên Huế',
       'Tỉnh Bình Dương', 'Tỉnh Bắc Ninh', 'Tỉnh Hậu Giang',
       'Tỉnh Quảng Ninh', 'Tỉnh Vĩnh Phúc', 'Hải Hưng', 'Tỉnh Bình Định',
       'Tỉnh Ninh Bình', 'Tỉnh An Giang', 'Tỉnh Thái Bình',
       'Tỉnh Hưng Yên', 'Hà Tây', 'Tỉnh Phú Yên', 'Tỉnh Lai Châu',
       'Tỉnh Vĩnh Long', 'Hà Bắc', 'Tỉnh Bạc Liêu', 'Nghĩa Định',
       'Tỉnh Ninh Thuận', 'Bình Trị Thiên', 'Tỉnh Trà V

In [22]:
# Lower case:
df['birth_place'] = df['noisinh'].str.lower()

# Loại bỏ ký tự đặc biệt
df['birth_place'] = df['birth_place'].str.replace(r'[-\\/@#$%^&*()_+={}\[\]:;"\'|<>,.?~`]', ' ', regex=True)
df['birth_place'] = df['birth_place'].str.replace('  ', ' ', regex=True)    

# Thay thế 'tp ' bằng 'thành phố ' trong cột 'noisinh'
df['birth_place'] = df['birth_place'].str.replace(r'\btp\b', 'thành phố', regex=True)

# Tạo danh sách major_cities
_cities = []

# Duyệt qua tất cả các giá trị trong cột 'birth_place' và kiểm tra xem có chứa 'thành phố' không
for location in df['birth_place']:
    try:
        if 'thành phố' in location:
            city_name = location.replace('thành phố ', '').strip()
            if city_name not in _cities:
                _cities.append(city_name)
    except:
        pass

# Định nghĩa hàm chuẩn hóa các giá trị trong cột 'birth_place'
def normalize_location(location, cities):
    try:
        location = location.strip()
        for city in cities:
            if city in location:

                return 'thành phố ' + city
        return location
    except:
        pass

df['birth_place'] = df['birth_place'].apply(lambda x: normalize_location(x, _cities))

print("Danh sách các thành phố lớn:", _cities)
print("\nCác giá trị duy nhất trong cột 'birth_place':")

df['birth_place'].unique()

Danh sách các thành phố lớn: ['hồ chí minh', 'hải phòng', 'hà nội', 'đà nẵng', 'cần thơ']

Các giá trị duy nhất trong cột 'birth_place':


array(['sông bé', 'tỉnh bến tre', 'thành phố hồ chí minh',
       'tỉnh bình thuận', 'tỉnh đồng nai', 'tỉnh kiên giang',
       'tỉnh lâm đồng', None, 'cửu long', 'tỉnh tây ninh',
       'tỉnh quảng ngãi', 'tỉnh thanh hóa', 'tỉnh long an',
       'tỉnh đồng tháp', 'tỉnh nghệ an', 'tỉnh hà tĩnh',
       'tỉnh bà rịa  vũng tàu', 'thành phố hải phòng', 'tỉnh quảng nam',
       'tỉnh nam định', 'tỉnh khánh hòa', 'tỉnh tiền giang',
       'tỉnh bắc giang', 'tỉnh quảng bình', 'tỉnh đắk lắk',
       'tỉnh gia lai', 'nghĩa bình', 'thành phố hà nội',
       'thành phố đà nẵng', 'tỉnh quảng trị', 'tỉnh thừa thiên huế',
       'tỉnh bình dương', 'tỉnh bắc ninh', 'tỉnh hậu giang',
       'tỉnh quảng ninh', 'tỉnh vĩnh phúc', 'hải hưng', 'tỉnh bình định',
       'tỉnh ninh bình', 'tỉnh an giang', 'tỉnh thái bình',
       'tỉnh hưng yên', 'hà tây', 'tỉnh phú yên', 'tỉnh lai châu',
       'tỉnh vĩnh long', 'hà bắc', 'tỉnh bạc liêu', 'nghĩa định',
       'tỉnh ninh thuận', 'bình trị thiên', 'tỉnh trà v

### 'gioitinh'
- Kiểm tra các loại giới tính sinh viên

In [27]:
print(check_missing_data(df, ['gioitinh']))
df['gioitinh'].value_counts()

Không có dữ liệu bị thiếu trong các cột: gioitinh
Empty DataFrame
Columns: [STT, masv, ngaysinh, gioitinh, noisinh, lopsh, khoa, hedt, nganh, tinhtrang, dantoc, tongiao, xuatthan, doituong, doituong_chitiet, khuvuc, vungmien, ngayvaodoan, ngayvaodang, diachi_phuongxa, diachi_quanhuyen, diachi_tinhtp, diachi_tamtru, diachi_tamtru_quanhuyen, diachi_tamtru_tinhtp, thanhtich_khenthuong, kyluat, quatrinh_congtac, chucvu_caonhat, bhyt, baoho_hoten, baoho_diachi, ghichu, baotin_diachi, bietdentu, bietdentu_other, lydohoc, lydohoc_other, ngaysinh_prc, birth_place]
Index: []


gioitinh
Nam    15610
Nữ      2315
Name: count, dtype: int64

### create 'nienkhoa'
- Tạo thuộc tính 'nienkhoa' dựa trên thông tin của mã số sinh viên 'masv'

In [53]:
df['nienkhoa'] = (df['masv'].astype(str).str[:-6]).astype(int) + 2000

# Di chuyển cột nienkhoa lên trước cột lopsh
cols = list(df.columns)
cols.insert(cols.index('lopsh'), cols.pop(cols.index('nienkhoa')))
df = df[cols]

### 'khoa'

In [31]:
print('số lượng sinh viên theo từng khoa: \n', df['khoa'].value_counts())
print(df['khoa'].unique())

số lượng sinh viên theo từng khoa: 
 khoa
MMT&TT    3900
HTTT      3690
CNPM      3137
KHMT      2858
KTMT      2416
KTTT      1924
Name: count, dtype: int64
['KHMT' 'HTTT' 'MMT&TT' 'KTMT' 'CNPM' 'KTTT']


### 'hedt'

In [32]:
print('Số lượng sinh viên theo hệ đào tạo: \n',df['hedt'].value_counts())
print(df['hedt'].unique())

Số lượng sinh viên theo hệ đào tạo: 
 hedt
CQUI    12474
CLC      4258
CTTT      591
CNTN      434
KSTN      168
Name: count, dtype: int64
['CQUI' 'CTTT' 'CNTN' 'KSTN' 'CLC']


### 'nganh'

In [35]:
print('Số lượng sinh viên không có thông tin \'nganh\': ', check_missing_data(df, ['nganh']))
df['nganh'].unique()

Không có dữ liệu bị thiếu trong các cột: nganh
Số lượng sinh viên không có thông tin 'nganh':  Empty DataFrame
Columns: [STT, masv, ngaysinh, gioitinh, noisinh, lopsh, khoa, hedt, nganh, tinhtrang, dantoc, tongiao, xuatthan, doituong, doituong_chitiet, khuvuc, vungmien, ngayvaodoan, ngayvaodang, diachi_phuongxa, diachi_quanhuyen, diachi_tinhtp, diachi_tamtru, diachi_tamtru_quanhuyen, diachi_tamtru_tinhtp, thanhtich_khenthuong, kyluat, quatrinh_congtac, chucvu_caonhat, bhyt, baoho_hoten, baoho_diachi, ghichu, baotin_diachi, bietdentu, bietdentu_other, lydohoc, lydohoc_other, ngaysinh_prc, birth_place]
Index: []


array(['D480101', 'D480104', 'D480102', 'D520214', 'D480103', 'D480201',
       'D480299', 'D52480104', 7480102, 7480109, '7480201_CLCN', 7480107],
      dtype=object)

In [34]:
df['nganh'].value_counts()

nganh
D480103         3137
D480104         3133
D480101         2832
D520214         2419
D480201         1718
D480102         1655
D480299         1277
7480102          870
D52480104        553
7480109          302
7480107           27
7480201_CLCN       2
Name: count, dtype: int64

### 'tinhtrang'

In [41]:
check_missing_data(df,['tinhtrang'])
print('Thống kê số lượng sinh viên theo tình trạng đào tạo: \n', df['tinhtrang'].value_counts())

Không có dữ liệu bị thiếu trong các cột: tinhtrang
Thống kê số lượng sinh viên theo tình trạng đào tạo: 
 tinhtrang
Đang học         6718
Đã tốt nghiệp    6637
Thôi học         3162
Tự do             704
Cảnh cáo          502
Buộc thôi học      76
Bảo lưu            73
Chuyển trường      27
Ngắn hạn           22
Gia hạn             4
Name: count, dtype: int64


### 'dantoc'

In [42]:
check_missing_data(df, ['dantoc'])
print('Thống kê số lượng sinh viên theo dân tộc: \n', df['dantoc'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột dantoc là: 8
Các dòng dữ liệu bị thiếu trong các cột dantoc là:
Thống kê số lượng sinh viên theo dân tộc: 
 dantoc
Kinh        17314
Hoa           324
Nùng           59
Khơ-me         55
Tày            53
Chăm           28
Mường          20
Ê-đê           12
Thái           12
Thổ             5
iGia-rai        5
Dao             4
Hmông           4
Sán Dìu         4
Chơ-ro          4
Cơ-ho           3
Chứt            2
Mnông           2
Cơ-tu           2
Hrê             1
Ra-glai         1
Xtiêng          1
Giáy            1
Chu-ru          1
Name: count, dtype: int64


### 'tongiao'

In [43]:
check_missing_data(df, ['tongiao'])
print('Thống kê số lượng sinh viên theo tôn giáo: \n', df['tongiao'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột tongiao là: 8
Các dòng dữ liệu bị thiếu trong các cột tongiao là:
Thống kê số lượng sinh viên theo tôn giáo: 
 tongiao
Không              14650
Phật giáo           1379
Thiên Chúa giáo     1357
Công giáo            350
Cao Đài               45
Hòa Hảo               43
Tin lành              39
Khác                  32
Bà La Môn             20
Hồi giáo               2
Name: count, dtype: int64


### 'xuatthan'

In [45]:
check_missing_data(df, ['xuatthan'])
print('Thống kê số lượng sinh viên dựa trên thông tin xuất thân: \n', df['xuatthan'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột xuatthan là: 1567
Các dòng dữ liệu bị thiếu trong các cột xuatthan là:
Thống kê số lượng sinh viên dựa trên thông tin xuất thân: 
 xuatthan
Khác                  8343
Nông dân              4876
Cán bộ - Công chức    2004
Công nhân              921
Quân nhân              139
Tư sản                  75
Name: count, dtype: int64


### 'doituong'

In [46]:
check_missing_data(df, ['doituong'])
print('Thống kê số lượng sinh viên dựa trên thông tin đối tượng: \n', df['doituong'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột doituong là: 17290
Các dòng dữ liệu bị thiếu trong các cột doituong là:
Thống kê số lượng sinh viên dựa trên thông tin đối tượng: 
 doituong
01-TS      233
06-TS      210
06-TB       42
06-CTB      40
01          21
02          21
06          17
06-BB       14
07          10
03           7
03-QNCA      7
04           2
04-CTB       2
04-TB        2
Không        2
03-TB        1
05-TN        1
04-HĐCM      1
04-LS        1
04-BB        1
Name: count, dtype: int64


### 'khuvuc'

In [47]:
check_missing_data(df,['khuvuc'])
print('Thống kê số lượng sinh viên theo khu vực: \n', df['khuvuc'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột khuvuc là: 3708
Các dòng dữ liệu bị thiếu trong các cột khuvuc là:
Thống kê số lượng sinh viên theo khu vực: 
 khuvuc
1        4209
2        4045
3        3150
2NT      2797
13          1
10          1
6           1
5           1
9           1
18          1
2-NT        1
7           1
12          1
11          1
15          1
20          1
19          1
8           1
14          1
Không       1
Name: count, dtype: int64


### 'diachi_tinhtp'

In [48]:
check_missing_data(df,['diachi_tinhtp'])
print('Thống kê số lượng sinh viên theo tỉnh thành phố: \n', df['diachi_tinhtp'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột diachi_tinhtp là: 158
Các dòng dữ liệu bị thiếu trong các cột diachi_tinhtp là:
Thống kê số lượng sinh viên theo tỉnh thành phố: 
 diachi_tinhtp
Thành phố Biên Hòa               519
Quận Gò Vấp                      357
Quận Bình Thạnh                  270
Thành phố Hồ Chí Minh            254
Quận Tân Bình                    248
Huyện Châu Thành                 244
Thành phố Nha Trang              231
Thành phố Buôn Ma Thuột          211
Quận 12                          209
Thành phố Vũng Tàu               190
Quận Thủ Đức                     172
Quận Tân Phú                     158
Thành phố Pleiku                 158
Thành phố Quy Nhơn               158
Tỉnh Đồng Nai                    149
Quận 9                           145
Quận Bình Tân                    139
Thành phố Đà Lạt                 134
Thị xã An Nhơn                   134
Thành phố Tuy Hòa                128
Huyện Hóc Môn                    120
Thành phố Quảng Ngãi             

### 'diachi_tamtru'

In [49]:
check_missing_data(df,['diachi_tamtru'])
print('Thống kê số lượng sinh viên theo địa chỉ tạm trú: \n', df['diachi_tamtru'].value_counts())

Số lượng dòng dữ liệu bị thiếu trong các cột diachi_tamtru là: 1106
Các dòng dữ liệu bị thiếu trong các cột diachi_tamtru là:
Thống kê số lượng sinh viên theo địa chỉ tạm trú: 
 diachi_tamtru
Thành phố Hồ Chí Minh                                                                                3064
Tỉnh Đồng Nai                                                                                        1209
Tỉnh Đắk Lắk                                                                                          884
Tỉnh Bình Định                                                                                        781
Tỉnh Lâm Đồng                                                                                         687
Tỉnh Quảng Ngãi                                                                                       552
Tỉnh Bà Rịa - Vũng Tàu                                                                                537
Tỉnh Khánh Hòa                                                    

## Feature_selection

In [54]:
select_cols = ['masv','ngaysinh_prc','birth_day','birth_month','birth_year','birth_place','gioitinh','nienkhoa','khoa','hedt','nganh','tinhtrang','dantoc','tongiao','xuatthan','doituong','khuvuc','diachi_tinhtp','diachi_tamtru']
data = df[select_cols]

# Save Data as ver 1

In [56]:
# Đường dẫn thư mục đã tạo sẵn
folder_path = os.path.join('data', 'ver_1')

# Kiểm tra xem thư mục có tồn tại hay không
if not os.path.exists(folder_path):
    print(f"Thư mục '{folder_path}' không tồn tại.")
else:
    # Lưu DataFrame vào file CSV trong thư mục 'ver_1'
    file_path = os.path.join(folder_path, 'students.csv')
    df.to_csv(file_path, index=False)
    print(f"DataFrame đã được lưu vào '{file_path}'")

DataFrame đã được lưu vào 'data\ver_1\students.csv'


## Data Quality Assessment

In [15]:
# current_year là năm hiện tại
current_year = 2023

# Chọn các cột cần thiết để kiểm tra tính đầy đủ
required_columns = ['masv', 'ngaysinh', 'gioitinh', 'lopsh', 'khoa', 'hedt', 'nganh']

# Áp dụng các hàm đánh giá cho từng dòng
df['completeness'] = df.apply(check_completeness, axis=1, required_columns=required_columns)
df['consistency'] = df.apply(check_consistency, axis=1)
df['timeliness'] = df.apply(check_timeliness, axis=1, current_year=current_year)
df['validity'] = df.apply(check_validity, axis=1)

In [16]:
attributes = ['masv']

# Kiểm tra tính duy nhất của các dòng
df['uniqueness'] = df.duplicated(subset=attributes, keep=False).astype(int)
df['uniqueness'] = 1 - df['uniqueness']  # Đảo ngược giá trị để 1 là duy nhất và 0 là trùng lặp


In [17]:
# Tính điểm trung bình cho từng dimension
completeness_score = df['completeness'].mean()
consistency_score = df['consistency'].mean()
timeliness_score = df['timeliness'].mean()
validity_score = df['validity'].mean()

# Đánh giá tính duy nhất cho toàn bộ DataFrame
uniqueness_score = check_uniqueness(df)

TypeError: check_uniqueness() missing 1 required positional argument: 'column'

In [ ]:
# Danh sách các dimension và trọng số tương ứng
dimensions = ['completeness', 'consistency', 'timeliness', 'validity', 'uniqueness']
weights = {'completeness': 0.2, 'consistency': 0.2, 'timeliness': 0.2, 'validity': 0.2, 'uniqueness': 0.2}

# Tính điểm chất lượng dữ liệu cho mỗi dòng
df['data_quality_score'] = df.apply(lambda row: np.average([row[dim] for dim in dimensions], weights=[weights[dim] for dim in dimensions]), axis=1)

# Tính điểm chất lượng dữ liệu cho toàn bộ tệp dữ liệu
overall_data_quality_score = np.mean(df['data_quality_score'])

In [ ]:
print(overall_data_quality_score)

0.999442119944212


In [ ]:
quality_scores = pd.DataFrame({
  'Total Rows': [df.shape[0]],
  'Total Features': [df.shape[1]],
  'Completeness (%)': [completeness_score*100],
  'Consitency (%)': [consistency_score*100],
  'Validity (%)': [validity_score*100],
  'Uniqueness (%)': [uniqueness_score*100],
  'Timeliness (%)': [timeliness_score*100]
})

print(quality_scores)

   Total Rows  Total Features  Completeness (%)  Consitency (%)  Validity (%)  \
0       17925              14          99.86053           100.0         100.0   

   Uniqueness (%)  Timeliness (%)  
0           100.0        99.86053  


In [ ]:
# Lưu kết quả vào file CSV
quality_scores.to_csv('report/dssinhvien_DQ_score.csv', index=False)
df.to_csv('report/dssinhvien_scored.csv', index=False)